In [ ]:
# Importing libraries.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Data Preprocessing

### Importing the data

In [ ]:
# Importing the dataset.
df = pd.read_csv(r'C:\Users\RaviTeja\Desktop\Great Learning\Capstone_project\loan.csv')
# set the max columns to none
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_1 = df.copy()

In [ ]:
df_1.shape

In [ ]:
df_1.info(verbose = True)

In [ ]:
# 
df_1.head()

In [ ]:
## removing the columns that do not add any importance to the dataset.

In [ ]:
# Id, member_id are unique numbers hence removing them.
# url -- removing as it is not useful for our analysis.
# desc -- removing as it is not useful for our analysis.
# zip-code -- zip-code provided was not complete.
df_1.drop(['id','member_id','url','desc','zip_code'],axis = 1,inplace = True)

In [ ]:
df_1.head()

In [ ]:
# We are trying to predict if the loan can be approved or not to the customer 
# but We have few columns which have data post loan approval.
# removing the columns that have data post approval.

# Columns:
# 1 "delinq_2yrs",
# 2 "revol_bal"
# 3 "out_prncp"
# 4 "total_pymnt"
# 5 "total_rec_prncp"
# 6 "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt", "next_pymnt_d", "chargeoff_within_12_mths"



In [ ]:
df_1.drop(columns = ["delinq_2yrs", "revol_bal", "out_prncp", "total_pymnt", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt", "next_pymnt_d", "chargeoff_within_12_mths"],axis = 1,inplace = True)

In [ ]:
df_1.dtypes

In [ ]:
# Cheking the data distribution
df_1.describe()

In [ ]:
# after checking the standard deviation of the data we can see that the pub_rec_bankruptcies have very less standard deviation among the data so removing the column
# also many columns are null which we will adress in the next step.

In [ ]:
df_1['pub_rec_bankruptcies'].value_counts() # as shown most of the data is 0.0

In [ ]:
df_1.drop(columns = ['pub_rec_bankruptcies'],axis = 1,inplace = True)

In [ ]:
# Checking the null values

In [ ]:
# Checking the null values present in the dataset.
df_1.isnull().sum()/len(df_1) *100

In [ ]:
# lets drop the columns that havee more than 50% of the data as null values

In [ ]:
df_null_cols = df_1.isnull().sum()/len(df_1) *100


In [ ]:
df_cols = df_null_cols[df_null_cols < 50].index # getting only the columns that have null values less than 50%
    

In [ ]:
df_2 = df_1[df_cols] # creating a final dataset with columns only having less than 50% of the dataset as null values

In [ ]:
df_2.sample(10)

In [ ]:
df_2.shape

In [ ]:
# Lets check at the datatypes for the columns
df_2.dtypes

In [ ]:
# df_2['term'] # we can see that this columns can be made to numeric by removing the months in the column data.
# df_2['int_rate'] # the percentage sign can be removed from the column data
# df_2['emp_length'] # the values are having < and + symbols hence it was treated as object
# df_2['issue_d'].unique() # can split the data into month and  year.
# df_2['loan_status'].unique() # current status wont be effective for us to determine if the customer will pay the debt or not
# df_2['last_credit_pull_d'].unique() # can split the data into month and year 
# df_2['revol_util'] # remove the percentages

In [ ]:
# df_2['term'] # we can see that this columns can be made to numeric by removing the months in the column data.
df_2['term'] = df_2['term'].str.strip(' months').astype(int)
df_2['revol_util'] = df_2['revol_util'].str.strip('%').astype(float)
df_2['int_rate'] = df_2['int_rate'].str.strip('%').astype(float)
df_2['employee_years_of_service'] = df_2['emp_length'].str.strip(' years') # renaming the column as employee_years_of_service for better understanding

In [ ]:
df_2.info()

In [ ]:
# df_2['issue_d'].unique() # can split the data into month and  year.
df_2['issue_d'].str.split('-')

In [ ]:
df_2['issue_date_month'] = pd.to_datetime(df['issue_d'],format='%b-%y').dt.strftime('%m').astype(int) # %b - month abbrevation ,
df_2['issue_date_year'] = pd.to_datetime(df['issue_d'],format = '%b-%y').dt.strftime('%Y') .astype(int)
#_______________________________
df_2['last_credit_pull_year']  = pd.to_datetime(df_2['last_credit_pull_d'],format = '%b-%y').dt.strftime('%Y')
df_2['last_credit_pull_year'] = df_2['last_credit_pull_year'].astype(float).astype('Int64')
#__________________________________
df_2['last_credit_pull_month'] = pd.to_datetime(df_2['last_credit_pull_d'],format = '%b-%y').dt.strftime('%m')
df_2['last_credit_pull_month']  =df_2['last_credit_pull_month'].astype(float).astype('Int64')

In [ ]:
df_2.drop(columns = ['issue_d'],axis = 1 , inplace = True)

In [ ]:
df_2['loan_status'].unique()

In [ ]:
df_2 = df_2[(df_2['loan_status'] != 'Current')]

In [ ]:
df_2['loan_status'].unique()

In [ ]:
df_2['emp_length'].unique() # removing the years from the data , if the data has <1 we will make it 0 and if it is 10+ we will make it 10.
df_2['employee_years_of_service'] = df_2['emp_length'].str.strip(' years') # renaming the column as employee_years_of_service for better understanding

In [ ]:
def symbol(x):
    if '<' in x:
        return 0
    elif '+' in x:
        year= int(x.split('+')[0])
        return year
    elif x == 'nan':
        pass
    else:
        year=int(x.split(' ')[0])
        return year
    

df_2['employee_years_of_service']=df_2['employee_years_of_service'].apply(lambda a:symbol(str(a)))


In [ ]:
df_2['employee_years_of_service'] = df_2['employee_years_of_service'].astype(float)

In [ ]:
df_2.drop(['emp_length'],axis = 1,inplace = True)

In [ ]:
# df_2['last_credit_pull_d']

In [ ]:
df_2['last_credit_pull_month'] = pd.to_datetime(df_2['last_credit_pull_d'],format = '%b-%y').dt.strftime('%m')
df_2['last_credit_pull_month']  =df_2['last_credit_pull_month'].astype(float).astype('Int64')

In [ ]:
df_2['last_credit_pull_year']  = pd.to_datetime(df_2['last_credit_pull_d'],format = '%b-%y').dt.strftime('%Y')
df_2['last_credit_pull_year'] = df_2['last_credit_pull_year'].astype(float).astype('Int64')

In [ ]:
df_2.drop(['last_credit_pull_d'],axis =1,inplace = True)

In [ ]:
df_2['revol_util'].head(5)

In [ ]:
df_2.info()

In [ ]:
df_2.info()

In [ ]:
df_2['earliest_cr_line_month'] = pd.to_datetime(df_2['earliest_cr_line'],format = '%b-%y').dt.strftime('%m').astype(float).astype('Int64')
df_2['earliest_cr_line_year'] = pd.to_datetime(df_2['earliest_cr_line'],format = '%b-%y').dt.strftime('%Y').astype(float).astype('Int64')

In [ ]:
df_2.drop(['earliest_cr_line'],axis = 1,inplace = True)

In [ ]:
df_2.info()

In [ ]:
# Dealt with all the columns that were needed to be changed to int and float.

In [ ]:
# Check how the data is distributed.
unq = pd.DataFrame()
unq['features'] = df_2.nunique().index
unq['values'] = df_2.nunique().values

In [ ]:
unq.sort_values(by = ['values'])

In [ ]:
# there are so many columns with only single value so they donot add any significance to our model.
# removing the single valued columns
df_2.drop(['collections_12_mths_ex_med',
'initial_list_status',
'policy_code',
'application_type',
'acc_now_delinq',
'delinq_amnt',
'pymnt_plan',
'out_prncp_inv',
'tax_liens'],axis = 1,inplace = True)

In [ ]:
df_2.info()

In [ ]:
df_2.isnull().sum()/len(df_1) *100

In [ ]:
# we null values in 
# employee_years_of_service - 2%
# emp_title - 6%
## -- will remove the null values that are less than 1% and try to fill the null values with 2% and 6% with mean or median in case of numerical variables and mode in case of categorical variables.
# title <1%
# last_credit_pull_year  - <1 %
# last_credit_pull_year- <1%
# revol_util  - <1%

In [ ]:
# Before filling the data checking the skewness of the numerical data.

In [ ]:
df_2['employee_years_of_service'].skew()

In [ ]:
sns.distplot(df_2['employee_years_of_service'])

In [ ]:
# The data is slightly right skewed and is mesokurtick
# Going with median for filling the null values for employee_years_of_service feature.

In [ ]:
df_2['employee_years_of_service'] = df_2['employee_years_of_service'].fillna(df_2['employee_years_of_service'].median())

In [ ]:
df_2.isnull().sum()/len(df_1) *100

In [ ]:
df_2['emp_title'] = df_2['emp_title'].fillna(df_2['emp_title'].mode())

In [ ]:
df_2.isnull().sum()/len(df_1) *100

In [ ]:
df_2['emp_title'] = df_2['emp_title'].fillna(df_2['emp_title'].mode()[0])

In [ ]:
df_2.isnull().sum()/len(df_1) *100

In [ ]:
# removing the rest of the null values as there are less than 1 % null values

df_2.dropna(inplace = True) # by default the how = 'any' and axis = 0 (rows)

In [ ]:
df_2.isnull().sum()/len(df_1) *100

In [ ]:
# There are no null values anymore.
df_2.shape #we have 30 columns and 38514 rows
# making this as the final dataset


In [ ]:
df_2.shape

In [ ]:
# lets check for the outliers in the data.

In [ ]:
cat_cols = df_2.select_dtypes(include=  'object').columns
num_cols = df_2.select_dtypes(include = ('int','float','int64')).columns

In [ ]:
cat_cols

In [ ]:
num_cols

In [ ]:
df_2.info()

In [ ]:
plt.figure(figsize= (35,5))
df_2[num_cols].boxplot()

In [ ]:
# We can see alot of outliers in the dataset.
# we can remove the outliers using the IQR.
df_2['funded_amnt_inv'].quantile(0.75)
df_2['funded_amnt_inv'].quantile(0.25)

In [ ]:
df_2.skew().sort_values(ascending = False)

In [ ]:
# annual_inc has more outliers 
# and pub_rec
# so we will remove those outliers and have the restr of the data left alone.

In [ ]:
df_3=  df_2.copy()

In [ ]:
df_3.head()

In [ ]:

Q3 = df_3['annual_inc'].quantile(0.75)
Q1 = df_3['annual_inc'].quantile(0.25)
IQR = Q3-Q1
df_3 = df_3[(df_3['annual_inc']>(Q1-1.5*IQR)) &  (df_3['annual_inc']<(Q3+1.5*IQR))] # getting the data into new column so that we will have backup of the data 

In [ ]:
df_3.head()

In [ ]:
df_3['annual_inc'].skew()

In [ ]:
df_3.skew()

In [ ]:
# most of the data are '0's so removing the column.
df_3['pub_rec'].value_counts()

In [ ]:
df_3.drop('pub_rec',axis = 1,inplace = True)

In [ ]:
df_3.dtypes

In [ ]:
df = df_3.copy()

In [ ]:
df_3.head()

# Exploratory Data Analysis

# UNIVARIATE & BIVARIATE ANALYSIS

### Categorical variables present after data cleaning

In [ ]:
df.select_dtypes('object').columns

### Numerical variables present after data cleaning

In [ ]:
df.select_dtypes(include=np.number).columns

## Distribution of Target variable

In [ ]:
# # Target variable

fig,axes=plt.subplots(1,2,figsize=(10,5))
d1=df['loan_status'].value_counts()
axes[0].bar(d1.index,d1.values)
axes[1].pie(d1.values,labels=d1.index,autopct='%0.2f%%')
axes[0].set_title('Loan status',fontsize=15)
axes[1].set_title('Pecentage of loan status',fontsize=15)

plt.show()

1. From the above plot , we ca say that 14.46 % people are the loan defaulters.
2. There are around 5232 people of total 36194 people who are loan defaulters.

## Convert target variable into numerical variable

In [ ]:
# target variable converted into 1's & 0''s
# loan defaulters are charged off people therefore charged off=1
# 0 : Fully paid
# 1 : Charged off

df['loan_status']=df['loan_status'].replace({'Fully Paid':0,'Charged Off':1})


## Loan Purpose

In [ ]:
# Loan purpose Vs Loan defaulters

count=df['purpose'].value_counts()
labels=count.index
values=count.values

#Percentage of loan purpose
plt.figure(figsize=(12,10))
plt.pie(x=values,labels=labels,autopct='%0.2f%%')
plt.title('\nLoan Purpose Information',fontsize=20)
plt.legend(labels=labels , loc=(1.2,0.7))
plt.show()


#Loan defaults w.r.t Purpose of loan
plt.figure(figsize=(15,6))
sns.barplot(x=df['purpose'] ,y=df['loan_status'])
plt.title('\nLoan defaulters w.r.t Purpose\n',fontsize=15)
plt.xticks(rotation=60)

plt.show()

1. Loans are taken for majority for debt consolidation, Other,Home improvement , major purchase & small buisiness purpose.
2. From the bar plot, we can observe that the probability of persong being loan defaulter is more in small buisiness,
Defaulter rate is more when person take loan for small business purpose.
3. Defaulter Rate will depend on the Purpose of the loan

## Term

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
# Term vs loan defaulters
fig,ax=plt.subplots(1,2  , figsize=(10,6))

#Countplot for Term
sns.countplot(df['term'] ,ax=ax[0] )
#Loan defaults vs term
sns.barplot(x=df['term'] ,y=df['loan_status'] ,ax=ax[1])
ax[0].set_title('\nTERM (in Months)\n',fontsize=15)
ax[1].set_title('\nLoan defaults % w.r.t TERM (in Months)\n',fontsize=15)
fig.tight_layout()


1. In the givan dataset, we are having more number of people with term 36 months than 60 months.
2. But The probability of loan getting defaulted is more for 60 months than 30 months.

## Grade

In [ ]:
#Grade 

fig,ax=plt.subplots(1,2 , figsize=(12,6))
# Countplot for grade
sns.countplot(df['grade'] ,ax=ax[0] ,order=df['grade'].value_counts().index)
# Barplot for loan defaults vs grade
sns.barplot(x=df['grade'] ,y =df['loan_status'] ,ax=ax[1] ,order=df['grade'].value_counts().index)
ax[0].set_title('\nGRADE\n',fontsize=15)
ax[1].set_title('\nLoan defaults % w.r.t Grade\n',fontsize=15)


1. There are more number of people with grade B.
2. Defaulter rate is high with the grade G and less for grade A.

## Sub-Grade

In [ ]:
# check for defaulters wrt subgrade in the data using countplot

#Countplot for sub grade
fig,ax=plt.subplots(2,1  , figsize=(15,12))
sns.countplot(x=df['sub_grade'] ,ax=ax[0],order=df['sub_grade'].value_counts().index)
#Loan defaulter vs sub grade
sns.barplot(x=df['sub_grade'] ,y=df['loan_status'] ,ax=ax[1] ,order=df['sub_grade'].value_counts().index)
ax[0].set_title('\n SUB GRADE\n',fontsize=20)
ax[1].set_title('\nLoan defaults % w.r.t SUB GRADE\n',fontsize=20)
plt.tight_layout()
plt.show()

1. There are more number of people with sub grade A4.
2. Defaulter rate is increasing with the sub grade , loan defaulter rate is more for F5 grade  and less for grade A1.
3. Sub Grade is useful for further analysis.

## Home ownership

In [ ]:
#Home ownership
fig,ax=plt.subplots(1,2  , figsize=(12,6))

#Countplot for home ownership
sns.countplot(df['home_ownership'] ,ax=ax[0] ,order=df['home_ownership'].value_counts().index)
#Barplot for loan defaults vs home ownership
sns.barplot(x=df['home_ownership'] ,y=df['loan_status'] ,ax=ax[1] ,order=df['home_ownership'].value_counts().index)
ax[0].set_title('\nHome Ownership\n',fontsize=15)
ax[1].set_title('\nLoan defaults % w.r.t Home Ownership\n',fontsize=15)
plt.tight_layout()

1. Loan defaulter rate is almost constant for all the home ownerships, slightly more for OTHER home ownership.
2. We can say that loan defaulters does not depends on home ownership.
3. Home ownership is not useful for further analysis.

In [ ]:
#Check for values count
df['home_ownership'].value_counts()

In [ ]:
# we will remove home ownership NONE
#Since Only one record is present for None category
df=df[df['home_ownership']!='NONE']

#Again check for value counts 
#df['home_ownership'].value_counts()

## Verification Status

In [ ]:
#Verification status
fig,ax=plt.subplots(1,2  , figsize=(12,6))

sns.countplot(df['verification_status'] ,ax=ax[0] ,order=df['verification_status'].value_counts().index)
sns.barplot(x=df['verification_status'] ,y=df['loan_status'] ,ax=ax[1] ,order=df['verification_status'].value_counts().index)
ax[0].set_title('\nVerification Status\n',fontsize=15)
ax[1].set_title('\nLoan defaults % w.r.t Verification Status\n',fontsize=15)
plt.tight_layout()
plt.show()

1. In the given data ,There are more records for which the ststus is non verified.
2. but the defaulter rate is more for verified status.

# Numerical variables

In [ ]:
df.select_dtypes(include=np.number).columns

## Interest rate

In [ ]:
#We have seen the distribution of intrest rate
df['int_rate'].describe()

In [ ]:
#Boxplot for interest rate
plt.figure(figsize=(10,6))
df['int_rate'].plot(kind='box')


In [ ]:
# Divide interest rate into groups based on intervals
df['int_rate_cat']=pd.cut(df['int_rate'],  [0,5,10,15,20,25,30], labels=['0-5','5-10','10-15','15-20','20-25','25-30'])

In [ ]:
#Interest rate

fig,ax=plt.subplots(1,2  , figsize=(12,6))
#Countplot for intrest rate categories
sns.countplot(df['int_rate_cat'] ,ax=ax[0] )
#Barplot for loan defaulters vs intrest rate categories
sns.barplot(x=df['int_rate_cat'] ,y=df['loan_status'] ,ax=ax[1] )
ax[0].set_title('\nInterest rate Range\n',fontsize=15)
ax[1].set_title('\nLoan defaults % w.r.t Interest rate range\n',fontsize=15)
plt.tight_layout()
plt.show()

1. There are more records with intrest rate between 10-15%
2. The rate of loan defaulter is more for highest intrest rate & less for lowest intrest rate.

## Annual income

In [ ]:
#We have seen the distribution of intrest rate
df['annual_inc'].describe()

In [ ]:
#Boxplot for annual income
plt.figure(figsize=(10,6))
df['annual_inc'].plot(kind='box')

plt.text(1.1, df['annual_inc'].max() , s=df['annual_inc'].max())
plt.text(1.1, df['annual_inc'].min() , s=df['annual_inc'].min())
plt.text(1.1, df['annual_inc'].quantile(0.25) , s=df['annual_inc'].quantile(0.25))
plt.text(1.1, df['annual_inc'].quantile(0.50) , s=df['annual_inc'].quantile(0.50))
plt.text(1.1, df['annual_inc'].quantile(0.75) , s=df['annual_inc'].quantile(0.75))

plt.ylabel('Annual Income',fontsize=15)
plt.show()
plt.show()

1. The median income is 58000 ,the minum income is 4000 & the maximum income is 234996.

In [ ]:
# Divide annual income into groups based on intervals
def annual_income(inc):
    if inc <= 50000:
        return 'low'
    elif inc > 50000 and inc <=100000:
        return 'medium'
    elif inc > 100000 and inc <=150000:
        return 'high'
    else:
        return 'very high'

df['annual_inc_cat'] = df['annual_inc'].apply(lambda x: annual_income(x))
df['annual_inc_cat'].value_counts()

In [ ]:
fig,ax=plt.subplots(1,2  , figsize=(12,6))
#Countplot for annual income range
sns.countplot(df['annual_inc_cat'] ,ax=ax[0] )
#Loan defaulter Vs annual income range
sns.barplot(x=df['annual_inc_cat'] ,y=df['loan_status'] ,ax=ax[1] )
ax[0].set_title('\nAnnual Income Range\n',fontsize=15)
ax[1].set_title('\nLoan defaults % w.r.t Annual income range\n',fontsize=15)
plt.tight_layout()
plt.show()

1. Defaulter rate is increasing with the annual income value, defaulter rate will depend on loan amount
2. The annual income variable is useful for further analysis.

## Loan amount

In [ ]:
df['loan_amnt'].describe()

In [ ]:
#Boxplot for loan amount
plt.figure(figsize=(10,6))
df['loan_amnt'].plot(kind='box')
plt.show()

The median loan amount is 9600 , the minimum loan amount is 500, the maximum loan amount is 35000

In [ ]:
# Divide Loan amount into groups based on intervals
def loan_amount(amt):
    if amt <= 5500:
        return 'low'
    elif amt > 5500 and amt <=10000:
        return 'medium'
    elif amt > 10000 and amt <=15000:
        return 'high'
    else:
        return 'very high'

df['loan_amnt_cat'] = df['loan_amnt'].apply(lambda x: loan_amount(x))
df['loan_amnt_cat'].value_counts()

In [ ]:
#Loan amount
fig,ax=plt.subplots(1,2  , figsize=(12,6))

#Countplot for loan amount range
sns.countplot(df['loan_amnt_cat'] ,ax=ax[0] )
#barplot for loan defaulters Vs loan amount range
sns.barplot(x=df['loan_amnt_cat'] ,y=df['loan_status'] ,ax=ax[1] )
ax[0].set_title('\nLoan amount Range\n',fontsize=15)
ax[1].set_title('\nLoan defaults % w.r.t Loan amount range\n',fontsize=15)
plt.tight_layout()
plt.show()

1. loan defaulter rate is increasing as loan amount range .
2. This feature is useful for further analysis

## Debt to Income ratio

In [ ]:
df['dti'].describe()

In [ ]:

df['dti_cat']=pd.cut(df['dti'],  [0,5,10,15,20,25,30], labels=['0-5','5-10','10-15','15-20','20-25','25-30'])


In [ ]:
#Debt to income ratio

fig,ax=plt.subplots(1,2  , figsize=(12,6))

sns.countplot(df['dti_cat'] ,ax=ax[0] )
sns.barplot(x=df['dti_cat'] ,y=df['loan_status'] ,ax=ax[1] )
ax[0].set_title('\nDebt to income ratio Range\n',fontsize=15)
ax[1].set_title('\nLoan defaults % w.r.t Debt to income ratio range\n',fontsize=15)
plt.tight_layout()
plt.show()

1. Loan defaulter rate is increasing with the debt to income ratio.
2. Dti is useful for finding the loan defaulter.

## Revolving line utilization rate

In [ ]:
#Boxplot for revol_util
plt.figure(figsize=(10,6))
df['revol_util'].plot(kind='box')

plt.show()

In [ ]:
# Divide revol utilization rate into groups based on intervals
df['revol_util_cat']=pd.cut(df['revol_util'],  [0,25,50,75,100], labels=['0-25','25-50','50-75','75-100'])

In [ ]:
#Loan defaulter vs revol util

plt.figure(figsize=(8,5))
sns.barplot(x=df['revol_util_cat'] ,y=df['loan_status'])
plt.show()

1. Loan defaulter rate is increasing with the revolving line utilization rate.
2. this is useful for finding the loan defaulter.

## Installments

In [ ]:
df['installment'].describe()

In [ ]:
# installment
def installment(n):
    if n <= 200:
        return 'low'
    elif n > 200 and n <=400:
        return 'medium'
    elif n > 400 and n <=600:
        return 'high'
    else:
        return 'very high'
    
df['installment_cat'] = df['installment'].apply(lambda x: installment(x))

In [ ]:
#Loan amount

fig,ax=plt.subplots(1,2  , figsize=(12,6))

sns.countplot(df['installment_cat'] ,ax=ax[0] )
sns.barplot(x=df['installment_cat'] ,y=df['loan_status'] ,ax=ax[1] )
ax[0].set_title('\nInstallment Range\n',fontsize=15)
ax[1].set_title('\nLoan defaults % w.r.t Installment range\n',fontsize=15)
plt.tight_layout()
plt.show()

1. Loan defaulter rate is increasing with the installment values.
2. Dti is useful for finding the loan defaulter.

### Installments Vs Loan amount

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot( df['loan_amnt'] ,df['installment'] )
plt.title('Installments Vs Loan amount' , fontsize=20)
plt.show()

1. It is obvious that if the loan amount is more, installment amount will also be more

### Annual income VS LOAN AMOUNT

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot( df['annual_inc'] ,df['loan_amnt'])
plt.title('Annual_inc Vs Loan amount' , fontsize=20)
plt.show()

1. People with annual income between 0 to 100000 tend to appply more for loan

# MULTIVARIATE ANALYSIS

In [ ]:
#Correlation matrix
plt.figure(figsize=(15,8))
sns.heatmap(df.corr(),annot=True)
plt.show()

1. There is a strong correlation between loan amount and funded amount.
2. There is a strong correlation between loan amount and funded amount invested.
3. There is a strong correlation between loan amount and installment.
4. There is a strong correlation between installement and funded_amount,funded_amount_invested.
5. There is a strong correlation between total_pymnt_inv and total_amount,funded_amount , funded_amnt_inv

# Pairplot




In [ ]:
df_num = df.select_dtypes(include = np.number)

### Loan amount & Interest rate 

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot( df['loan_amnt'] ,df['int_rate'] ,hue=df['loan_status'])
plt.title('Interest rate (%) Vs Loan amount' , fontsize=20)
plt.show()

The defaulters are present at all the places

In [ ]:
df_final = df.copy()

In [ ]:
df_final.sample(20)

In [ ]:
df_final.shape

In [ ]:
df_final.sample(20)

In [ ]:
# encoding the categorical features to 

In [ ]:
cat_col = df_final.select_dtypes(include = object).columns

In [ ]:
cat_cols

In [ ]:
df_cat = df_final[cat_cols]

In [ ]:
#Performing encoding for the columns
df_dummies=pd.get_dummies(df_cat[['home_ownership','grade','verification_status']],drop_first=True)
df_dummies.shape

In [ ]:
#Performing label encoding for Addr_state
state=df_cat['addr_state'].unique()
add_dict={}
j=1
for i in state:
    add_dict[i]=j
    j=j+1
df_cat['addr_state']=df_cat['addr_state'].map(add_dict)


In [ ]:
#Performing label encoding for purpose
purpose=df_cat['purpose'].unique()
purpose_dict={}
j=1
for i in purpose:
    purpose_dict[i]=j
    j=j+1
df_cat['purpose']=df_cat['purpose'].map(purpose_dict)    

In [ ]:
#Dropping columns home_ownership,verification_status,term_months,grade
df_cat.drop(columns=['home_ownership','grade','verification_status'],inplace=True)

In [ ]:
#concatenating the df_dummies, df_cat
df_cat=pd.concat([df_cat,df_dummies],axis=1)
df_cat.shape

In [ ]:
df_cat.sample(20)

In [ ]:
df_cat.drop(['emp_title','title','sub_grade'],axis = 1,inplace = True)

In [ ]:
df_cat.sample(10)

In [ ]:
df_num = df_final.select_dtypes(include = np.number)

# Feature selection 

In [ ]:
from sklearn.feature_selection import mutual_info_classif,mutual_info_regression
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(df_num.corr(),annot=True)

In [ ]:
#The features of loan_amnt, funded_amnt_inv, installment have high correlation. So dropping two columns among them

In [ ]:
#The features of loan_amnt, funded_amnt_inv, installment have high correlation. So dropping two columns among them
plt.figure(figsize=(15,8))
sns.heatmap(df_num.drop(columns=['loan_amnt','installment']).corr(),annot=True)

In [ ]:
plt.figure(figsize=(12,7))
sns.heatmap(df_num.drop(columns=['funded_amnt_inv','loan_amnt']).corr(),annot=True)

In [ ]:
y=df_final['loan_status']

In [ ]:
#Calculating the feature importance for categorical columns
feature_importances_cat=mutual_info_classif(df_cat,y)
features_importances_cat=pd.Series(feature_importances_cat,df_cat.columns[0:len(df_cat.columns)])
features_importances_cat.plot(kind='barh')
plt.show()

In [ ]:
#Concating the df_num,df_cat to a single dataframe
df2=pd.concat([df_num,df_cat],axis=1)
print(df2.shape)
df2.head()

In [ ]:
df2_final = df2.drop(['loan_status'],axis = 1)

In [ ]:
#Using Sequential Feature selection technique and applying forward substitution
lr=LogisticRegression(max_iter=5000)


In [ ]:
sfs_forward=SequentialFeatureSelector(lr,k_features=25)
sfs_forward.fit(df2_final,y)
sfs_forward.k_feature_names_

In [ ]:
#Using Sequential Feature selection technique and applying backward elimination
sfs_backward=SequentialFeatureSelector(lr,k_features=25,forward=False)
sfs_backward.fit(df2_final,y)
sfs_backward.k_feature_names_

In [ ]:
from sklearn.feature_selection import RFECV



rfecv = RFECV(
    estimator=LogisticRegression(),
    min_features_to_select=10,
    step=5,
    n_jobs=-1,
    scoring="r2",
    cv=5,
)   

rfecv_log = rfecv.fit(df2_final, y)

In [ ]:
len(df2_final.loc[:,rfecv_log.support_].columns)

In [ ]:
# best features to select are 23 
df2_final_rfe = df2_final.loc[:,rfecv_log.support_]

In [ ]:
features = df2_final_rfe.columns

In [ ]:
features

In [ ]:
features

# Scaling the features

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=df2_final_rfe
x_scaled = pd.DataFrame(sc.fit_transform(x),columns=x.columns)

In [ ]:
# The scaled data will only be used for logistic regression and KNN 
# as these algorithms are distance based .

In [ ]:
df2_final_rfe.info()

In [ ]:
df2_final_rfe.head()

In [ ]:
features

In [ ]:
df2_final_rfe.shape

# Balancing the dataset

The project on loan default model is very much analagous to the defect or disease capturing models where there will be always more data on OK and only fewer example data on NOT OK or default issues. Such data is the natural data on a practical real life situation and hence does not need tp be balanced.

## Train test split

In [ ]:
# non scaled data 
from sklearn.model_selection import train_test_split

Xtrain,Xtest,ytrain,ytest=train_test_split(df2_final_rfe , y , test_size=0.3 , random_state=10)

In [ ]:
# scaled data for logistic regression and KNN
Xtrain_scaled,Xtest_scaled,ytrain_sc,ytest_sc = train_test_split(x_scaled,y,test_size = 0.3,random_state = 10)

# Building the models

In [ ]:
pip install xgboost

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, RepeatedKFold
from sklearn.metrics import accuracy_score, auc, roc_auc_score,roc_curve,classification_report, confusion_matrix,f1_score 
from sklearn.metrics import precision_score,recall_score
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

# Logistic Regression

In [ ]:
# training data 
from sklearn.metrics import classification_report
logreg = LogisticRegression(random_state=10)
logreg.fit(Xtrain_scaled, ytrain_sc)
ypred_logre = logreg.predict(Xtrain_scaled)

print(classification_report(ytrain_sc, ypred_logre))


In [ ]:
log_reg = LogisticRegression(random_state=10)
log_reg.fit(Xtrain_scaled, ytrain_sc)
log_reg_train = log_reg.predict(Xtrain_scaled)

print("Train report\n\n",classification_report(ytrain_sc, log_reg_train))
print("Train f1_score \n\n",f1_score(ytrain_sc, log_reg_train),"\n")
log_reg_test = log_reg.predict(Xtest_scaled)
print("Test report\n\n",classification_report(ytest_sc, log_reg_test))
print("Test f1_score \n",f1_score(ytest_sc, log_reg_test))

In [ ]:
# trying to increase the accuracy

In [ ]:
param = {'penalty':['l1','l2','elasticnet']}
grid_log = GridSearchCV(LogisticRegression(random_state= 10),param_grid= param)

In [ ]:
grid_log_best = grid_log.fit(df2_final_rfe,y)

In [ ]:
grid_log_best.best_params_

In [ ]:
# by default logistic regression uses l2 penalty so this is the best f1_score we can get 

In [ ]:
# There is no hint of overfitting or underfitting of the model.
# so it is safe to go ahead and build out final logistic model

In [ ]:
# final model 
from sklearn.metrics import classification_report
logreg = LogisticRegression(random_state=10)
logreg.fit(Xtrain_scaled, ytrain_sc)
ypred_logreg = logreg.predict(Xtest_scaled)

print(classification_report(ytest_sc, ypred_logreg))


In [ ]:
ypred_logreg

In [ ]:
accuracy_logreg = accuracy_score(ytest, ypred_logreg)

In [ ]:
logreg = LogisticRegression(random_state=10)
logreg.fit(Xtrain_scaled, ytrain_sc)
ypred_logreg = logreg.predict(Xtest_scaled)
accuracy_logreg = accuracy_score(ytest_sc, ypred_logreg)
f1_logreg = f1_score(ytest_sc, ypred_logreg)
precision_logreg = precision_score(ytest_sc, ypred_logreg)
recall_logreg = recall_score(ytest_sc, ypred_logreg)
auc_logreg = roc_auc_score(ytest_sc, ypred_logreg)
conf_logreg = confusion_matrix(ytest_sc, ypred_logreg)
print('Confusion matrix:\n',conf_logreg)
print('Auc:',auc_logreg)
print('Recall:',recall_logreg)
print('Precision:',precision_logreg)
print('f1_score:',f1_logreg)
print('Accuracy:',accuracy_logreg)

In [ ]:
y_pred_prob_logreg = logreg.predict_proba(Xtest_scaled)
y_pred_prob_logreg1 = logreg.predict_proba(Xtrain_scaled)
fpr,tpr,th = roc_curve(ytest,y_pred_prob_logreg[:,1])
plt.plot([0,1],[0,1],'r--')
plt.plot(fpr, tpr)
plt.grid(True)
plt.show()

# Decision tree classifier

In [ ]:
# The overall model has not been very good in predicting the loan defaulter.
# the category 1 has a very less accuracy.
# now we can try different models and fine tune to the best model for this data.


In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier(random_state=10)

In [ ]:
dtc.fit(Xtrain,ytrain)

In [ ]:
pred_dtc = dtc.predict(Xtest)

In [ ]:
print(classification_report(ytest,pred_dtc))

In [ ]:
f1_logreg = f1_score(ytest,pred_dtc)

In [ ]:
f1_logreg

In [ ]:
# f1_score has increased drastically from 0.5 to 0.85 due to decision tree.

In [ ]:
# lets check for overfitting or underftitting of the model
dtc_train = DecisionTreeClassifier(random_state= 10)
dtc_train.fit(Xtrain, ytrain)
dtc_pred_train = dtc_train.predict(Xtrain)

print("Train report\n\n",classification_report(ytrain, dtc_pred_train))
print("Train f1_score \n\n",f1_score(ytrain, dtc_pred_train))
dtc_pred_test = dtc_train.predict(Xtest)
print("Test report\n\n",classification_report(ytest, dtc_pred_test))
print("Test f1_score \n",f1_score(ytest, dtc_pred_test))

In [ ]:
# The model is visible to be  overfitting


#### Cross validation on Decision Tree

In [ ]:
# Lets check how well the model is performing on unseen data using cross validation

In [ ]:
# We are performing a cross validation on non hypertuned Decision tree model

In [ ]:
from sklearn.model_selection import cross_val_score

cv_decision = cross_val_score(DecisionTreeClassifier(random_state = 10),df2_final_rfe,y,cv = 10)
# we are checking the cv score 10 fold

In [ ]:
cv_decision.mean()

In [ ]:
# The overall score from the decision tree is very high even with unseen data .
# This model is performing good even when the parameters are not tuned.

#### Grid Search on Decision Tree

In [ ]:
# Lets try if we can increase the f1_score of the model more.

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters_decision = {'criterion':["gini", "entropy"],'splitter':["best", "random"],'max_depth':[8,9,10,11,12,13,14,15,16],'min_samples_leaf':[2,3,4,5,6,7,8]}

gd_decision = GridSearchCV(estimator=DecisionTreeClassifier(random_state= 10),param_grid=parameters_decision,cv = 5)

In [ ]:
gd_decision.fit(df2_final_rfe,y)

In [ ]:
gd_decision.best_estimator_

In [ ]:
gd_decision.best_params_

In [ ]:
# These are the best parameters for decision tree that are providing the best score .
# lets try to fit the model with these paramters and see if the overall f1_score has increased.

In [ ]:
decision_grid = DecisionTreeClassifier(criterion="entropy",max_depth = 10 ,splitter = 'best',min_samples_leaf=6,random_state = 10)

In [ ]:
decision_grid.fit(Xtrain,ytrain)
y_pred_dec_grid = decision_grid.predict(Xtest)

In [ ]:
y_pred_dec_grid

In [ ]:
f1_score(ytest,y_pred_dec_grid)

In [ ]:
# The f1_score has increased by 4%.
# lets put cross validation on the hypertuned model and see how it performs on unseen data.

In [ ]:
from sklearn.model_selection import cross_val_score

cv_decision_grid = cross_val_score(DecisionTreeClassifier(criterion="entropy",max_depth = 10 ,splitter = 'best',min_samples_leaf =6,random_state = 10),df2_final_rfe,y,cv = 10)
# we are checking the cv score 10 fold

In [ ]:
cv_decision_grid

In [ ]:
cv_decision_grid.mean()

In [ ]:
# The mean of the cross validation has been increased by 2%
# lets fit the model and see  accuracy and recall score's of the hypertuned model.

In [ ]:
decision_grid_final = DecisionTreeClassifier(criterion="entropy",max_depth = 10 ,splitter = 'best',min_samples_leaf=6,random_state=10)
decision_grid_final.fit(Xtrain,ytrain)
ypred_grid_decision_final = decision_grid_final.predict(Xtest)

In [ ]:
print(classification_report(ytest,ypred_grid_decision_final))

In [ ]:
# Lets see if the model is overfitting or underfitting
# lets check for overfitting or underftitting of the model
dtc_grid_train = DecisionTreeClassifier(criterion="entropy",max_depth = 10 ,splitter = 'best',min_samples_leaf=6,random_state=10)
dtc_grid_train.fit(Xtrain, ytrain_sc)
dtc_grid_pred_train = dtc_train.predict(Xtrain)

print("Train report\n\n",classification_report(ytrain, dtc_grid_pred_train))
print("Train f1_score \n\n",f1_score(ytrain, dtc_grid_pred_train))
dtc_grid_pred_test = dtc_grid_train.predict(Xtest)
print("Test report\n\n",classification_report(ytest, dtc_grid_pred_test))
print("Test f1_score \n",f1_score(ytest, dtc_grid_pred_test))

In [ ]:
# The models overfitting has reduced a bit
# but the model is still visible to be overfitting.
# and is unable to generalise the given data.

In [ ]:
# Precision and recall have increased by a large margin when compared to base model (Logistic regression).

In [ ]:
# Since the model is overfitting can resolve this issue with regularization.
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier(alpha = 0.1,random_state=10)
ridge.fit(x_scaled,y)
ridge.score(x_scaled,y)

In [ ]:
cs_ridge = cross_val_score(ridge,x_scaled,y,cv = 10)
cs_ridge.mean()

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
ridgecv = RidgeClassifierCV(np.arange(0, 1, 0.01),cv = 10,)

In [ ]:
ridgecv.fit(x_scaled,y)

In [ ]:
ridgecv.alpha_

In [ ]:
ridgecv = RidgeClassifier(alpha = 0.66,)
ridgecv.fit(x_scaled,y)

In [ ]:
ridgecv.score(x_scaled,y)

In [ ]:
cs_ridge_cv = cross_val_score(ridgecv,x_scaled,y,cv = 10)
cs_ridge_cv.mean()
# best alpha value is 0.66 for ridgeclassifier

In [ ]:
ridgecv_1 = RidgeClassifier(alpha = 0.66,)
ridgecv_1.fit(Xtrain_scaled,ytrain_sc)
ridgecv_pred_1 = ridgecv_1.predict(Xtest_scaled)

In [ ]:
f1_score(ytest_sc,ridgecv_pred_1)

In [ ]:
len(ridgecv_1.feature_names_in_)

# KNN 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knear = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knear.fit(Xtest_scaled,ytest_sc)
y_pred_knn = knear.predict(Xtest_scaled)

In [ ]:
f1_score(ytest_sc,y_pred_knn)

In [ ]:
print(classification_report(ytest_sc,y_pred_knn))

## Testing overfitting and underfitting

In [ ]:
# training data 
from sklearn.metrics import classification_report
knn_train = KNeighborsClassifier(n_neighbors=5)
knn_train.fit(Xtrain_scaled, ytrain_sc)
ypred_knn = knn_train.predict(Xtrain_scaled)

print("Train classification report \n\n",classification_report(ytrain_sc, ypred_knn),"\n")
print("Train f1_score \n\n",f1_score(ytrain_sc, ypred_knn),"\n" )

y_pred_knn_test = knn_train.predict(Xtest_scaled)
print("Test classification report \n\n",classification_report(ytest_sc, ypred_knn_test),"\n")
print("Test f1_score \n\n",f1_score(ytest_sc, ypred_knn_test) )

## Grid search on KNN

In [ ]:
# Grid search

parameters_decision = {'n_neighbors' :  [1,2,3,4,5,6,7,8,9,10],'weights' : ['uniform','distance'],'p':[1,2]}

gd_knn = GridSearchCV(estimator=KNeighborsClassifier(),param_grid=parameters_decision)

In [ ]:
gd_knn.fit(x_scaled,y)

In [ ]:
gd_knn.best_params_

In [ ]:
# grid serach model

In [ ]:
knn_grid = KNeighborsClassifier(n_neighbors=10,p = 1,weights = 'distance')
knn_grid.fit(Xtest_scaled,ytest_sc)
y_pred_knn_grid = knn_grid.predict(Xtest_scaled)

In [ ]:
print(classification_report(ytest_sc,y_pred_knn_grid))

In [ ]:
f1_score(ytest_sc,y_pred_knn_grid)

## Checking overfitting and underfitting of Grid Search KNN

In [ ]:
knn_grid_train = KNeighborsClassifier(n_neighbors=10,p = 1,weights = 'distance')
knn_grid_train.fit(Xtrain_scaled, ytrain_sc)
ypred_knn = knn_grid_train.predict(Xtrain_scaled)

print("Train report\n\n",classification_report(ytrain_sc, ypred_knn))
print("Train f1_score \n\n",f1_score(ytrain_sc, ypred_knn))
ypred_knn_test = knn_grid_train.predict(Xtest_scaled)
print("Test report\n\n",classification_report(ytest_sc, ypred_knn_test))
print("Test f1_score \n",f1_score(ytest_sc, ypred_knn_test))

In [ ]:
# Grid search has increased the overfitting of the data.

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random = RandomForestClassifier()
random.fit(Xtrain,ytrain)
ypred_random = random.predict(Xtest)
print(classification_report(ytest,ypred_random))

In [ ]:
f1_score(ytest,ypred_random)

In [ ]:
random = RandomForestClassifier()
random.fit(Xtrain,ytrain)
ypred_random_train = random.predict(Xtrain)
print("train classification report\n\n",classification_report(ytrain,ypred_random_train),"\n")
print("train f1 score \n\n",f1_score(ytrain,ypred_random_train),"\n")

ypred_random_test = random.predict(Xtest)
print("test classification report \n\n",classification_report(ytest,ypred_random_test),"\n")
print("test f1 score",f1_score(ytest,ypred_random_test),"\n")

In [ ]:
# The model is overfitting 

In [ ]:
# Model comparisions

In [ ]:
accuracy_scaled = [accuracy_score(ytest_sc,ypred_logreg),accuracy_score(ytest_sc,y_pred_knn),accuracy_score(ytest_sc,y_pred_knn_grid)]
precision_scaled = [precision_score(ytest_sc,ypred_logreg),precision_score(ytest_sc,y_pred_knn),precision_score(ytest_sc,y_pred_knn_grid)]
recall_scaled = [recall_score(ytest_sc,ypred_logreg),recall_score(ytest_sc,y_pred_knn),recall_score(ytest_sc,y_pred_knn_grid)]
f1_score_scaled = [f1_score(ytest_sc,ypred_logreg),f1_score(ytest_sc,y_pred_knn),f1_score(ytest_sc,y_pred_knn_grid)]


In [ ]:
accuracy = [accuracy_score(ytest,pred_dtc),accuracy_score(ytest,ypred_grid_decision_final),accuracy_score(ytest,ypred_random)]
precision = [precision_score(ytest,pred_dtc),precision_score(ytest,ypred_grid_decision_final),precision_score(ytest,ypred_random)]
recall = [recall_score(ytest,pred_dtc),recall_score(ytest,ypred_grid_decision_final),recall_score(ytest,ypred_random)]
f1_score_1 = [f1_score(ytest,pred_dtc),f1_score(ytest,ypred_grid_decision_final),f1_score(ytest,ypred_random)]
# 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

gbc = GradientBoostingClassifier(random_state = 10)
gbc.fit(Xtrain_scaled,ytrain_sc)

In [ ]:
gbc_pred = gbc.predict(Xtest_scaled)

In [ ]:
print(classification_report(ytest_sc,gbc_pred))

In [ ]:
f1_score(ytest_sc,gbc_pred)

In [ ]:
gbc.fit(Xtrain_scaled,ytrain_sc)
gbc_pred_train = gbc.predict(Xtrain_scaled)
print(f1_score(ytrain_sc,gbc_pred_train))

gbc_pred_test = gbc.predict(Xtest_scaled)
f1_score(ytest_sc,gbc_pred_test)

In [ ]:
GradientBoostingClassifier()

In [ ]:
# Grid search on gradient boosting.
params_gradient = {'loss':['deviance', 'exponential'],'learning_rate':np.arange(0,1,0.01),'min_samples_split':[2,3,4,5,6,7,8,9,10],'min_samples_leaf':[2,3,4,5,6,7,8,9,10]}
gradient_grid = GridSearchCV(GradientBoostingClassifier(),param_grid=params_gradient)


In [ ]:
gradient_grid.fit(x_scaled,y)

In [ ]:
model_comparision_1 = pd.DataFrame(columns = ['Logistic_Regression','KNN','KNN_Grid'],index=  ['Accuracy','Precision','Recall','F1_score'])
model_comparision = pd.DataFrame(columns = ['Decision_Tree','Decision_Tree_Grid','Random_Forest'],index=  ['Accuracy','Precision','Recall','F1_score'])


In [ ]:

model_comparision_1.loc['Accuracy'] = accuracy_scaled
model_comparision_1.loc['Precision'] = precision_scaled
model_comparision_1.loc['Recall'] = recall_scaled
model_comparision_1.loc['F1_score'] = f1_score_scaled

In [ ]:
model_comparision.loc['Accuracy'] = accuracy
model_comparision.loc['Precision'] = precision
model_comparision.loc['Recall'] = recall
model_comparision.loc['F1_score'] = f1_score_1

In [ ]:
final_model_comparision = pd.concat([model_comparision,model_comparision_1],axis = 1)

In [ ]:
final_model_comparision

In [ ]:
# As per the model Decision tree is the best model
# with highest scores